In [ ]:
import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

### Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load transfection data from exp83.5 (iPS11 cells)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
exp83_5_path = base_path/'2024.04.29_exp83.5'/'export'

plates = pd.DataFrame({
    'data_path': [exp83_5_path],
    
    'yaml_path': [exp83_5_path/'exp83.5_wells.yaml'],
    
    'biorep': [1],
    
    'exp': ['exp83.5']
})

output_path = rd.rootdir/'output'/'iPSC'/'more'
cache_path = rd.rootdir/'output'/'iPSC'/'more'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata['miR_'] = (metadata['miR'].isin(['none','na'])).map({True: '–', False:'+'})
metadata['ts_'] = (metadata['ts'].isin(['none','na'])).map({True: '–', False:'+'})
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color'] # colored by design
construct_markers = metadata_dict['markers']

In [ ]:
# Create color palette for comparing promoters
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EF1a'), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='CAG'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EFS'), 'color'] = base.colors['red']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='hPGK'), 'color'] = base.colors['green']

metadata2.loc[(metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'].apply(base.get_dark_color).apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
promoter_palette = metadata_dict2['color']
promoter_markers = metadata_dict2['markers']

In [ ]:
# Create color palette for comparing designs
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='miR'), 'color'] = base.colors['purple']
metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'] = metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==1), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==2), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==3), 'color'] = base.colors['red']

metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'].apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'
metadata2.loc[(metadata2['group']=='ts3'), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='ts5'), 'markers'] = 'v'
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'markers'] = 'D'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'
metadata2.loc[(metadata2['group']=='ts5'), 'linestyle'] = '--'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
designs_palette = metadata_dict2['color']
designs_markers = metadata_dict2['markers']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

Gate data per experiment based on transfection marker expression

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

In [ ]:
# Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

In [ ]:
# Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')
display(stats)

In [ ]:
# Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

In [ ]:
# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(['construct','exp','biorep'])[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
display(fits)

### Distribution plots

In [ ]:
# EF1a
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (1e3,2e5)
ylim = (5e1,1e6)
biorep = 1
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design']<=1) & (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-design1.svg')), bbox_inches='tight')

In [ ]:
# EF1a
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (1e3,2e5)
ylim = (5e1,1e6)
biorep = 1
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,2])) & (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-design2.svg')), bbox_inches='tight')

In [ ]:
# EF1a
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (1e3,2e5)
ylim = (5e1,1e6)
biorep = 1
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['design'].isin([0,3])) & (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-design3.svg')), bbox_inches='tight')

In [ ]:
# CAG
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (1e3,2e5)
ylim = (5e1,1e6)
biorep = 1
plot_df = df_quantiles[(df_quantiles['group'].isin(['base','controller'])) & (df_quantiles['promoter']=='CAG') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_CAG.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (1e3,2e5)
ylim = (5e1,1e6)
biorep = 1
plot_df = df_quantiles[((df_quantiles['group']=='miR')) & (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miR.FF4','miR.FF5'])], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miRE.FF4','miRE.FF5'])], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miR-only.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (1e3,2e5)
ylim = (5e1,1e6)
biorep = 1
plot_df = df_quantiles[((df_quantiles['group'].isin(['ts3','ts5']))) & (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['group']=='ts5'], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['group']=='ts3'], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-ts-only.svg')), bbox_inches='tight')

### Stat plots

In [ ]:
# EF1a miRE-FF4 all designs
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (5e1,1e6)
plot_df = stats[((stats['group']=='controller') | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[((fits['group']=='controller') | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-designs.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF4 controls
fig, axes = plt.subplots(1,3, figsize=(15,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))
ylim = (5e1,1e6)
plot_df = stats[(stats['group'].isin(['base','miR','ts3','ts5'])) &
                (stats['promoter']=='EF1a')].copy()
plot_df.sort_values(['group','ts_kind'], inplace=True)
plot_df2 = fits[(fits['group'].isin(['base','miR','ts3','ts5'])) &
                (fits['promoter']=='EF1a')].copy()
plot_df2.sort_values(['group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', xlabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=True, ax=ax)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', xlabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), ylim=(0,1.2), ylabel='', xlabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['miR_loc','ts_loc'], annotate=False, ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-controls.svg')), bbox_inches='tight')

In [ ]:
# CAG
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (5e1,1e6)
plot_df = stats[(stats['promoter']=='CAG')].copy()
plot_df.sort_values(['group','ts_kind'], inplace=True)
plot_df2 = fits[(fits['promoter']=='CAG')].copy()
plot_df2.sort_values(['group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', xlabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', xlabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), ylim=(0,1.2), ylabel='', xlabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_CAG.svg')), bbox_inches='tight')

In [ ]:
# EF1a design 1
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
ylim = (5e1,1e6)
plot_df = stats[(stats['promoter']=='EF1a') & (stats['design']<=1) & (stats['group'].isin(['controller','base']))].copy()
plot_df.sort_values(['group','ts_kind'], inplace=True)
plot_df2 = fits[(fits['promoter']=='EF1a') & (fits['design']<=1)& (fits['group'].isin(['controller','base']))].copy()
plot_df2.sort_values(['group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', xlabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), ylim=ylim, ylabel='', xlabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), ylim=(0,1.2), ylabel='', xlabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-design1.svg')), bbox_inches='tight')